In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

In [ ]:
### Access the New York Times API

In [ ]:
# Set the base URL
nyt_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"


# Build URL parameters
nyt_params = {
    'fq': filter_query,
    'sort': sort,
    'fl': field_list,
    'begin_date': begin_date,
    'end_date': end_date,
    'api-key': nyt_api_key
}

# Function to build the query URL
def build_query_url(base_url, params):
    query_params = "&".join([f"{key}={value}" for key, value in params.items()])
    return base_url + query_params


# Print the first 5 rows of the reviews_list
if reviews_list:
    print("Preview of New York Times Data:")
    print(json.dumps(reviews_list[:5], indent=4))
else:
    print("No reviews were retrieved.")


# Convert reviews_list to a Pandas DataFrame using json_normalize()
if reviews_list:
    reviews_df = pd.json_normalize(reviews_list)
    reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
else:
    print("No reviews were retrieved.")

# Print the first 5 rows of the normalized DataFrame
print("Normalized DataFrame (first 5 rows):")
print(reviews_df.head())

# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].dropna().to_list()

# Preview the titles in a readable format
print("Preview of Titles List:")
for title in titles[:20]:  # Print the first 20 titles for readability
    print(f"'{title}'")

# Build the query URL
query_url = build_query_url(nyt_url, nyt_params)


In [ ]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19

    # create query with a page number
    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    # Add a twelve second interval between queries to stay within API query limits
for page in range(20):
    nyt_params['page'] = page
    query_params = "&".join([f"{key}={value}" for key, value in nyt_params.items()])
    query_url = nyt_url + query_params
    response = requests.get(query_url)
    time.sleep(12)
# Try and save the reviews to the reviews_list
 # loop through the reviews["response"]["docs"] and append each review to the list
 # Print the page that was just retrieved
# Print the page number that had no results then break from the loop
    
    try:
        reviews = response.json()
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        print(f"Successfully retrieved page {page}")
    except Exception as e:
        print(f"No results on page {page}, exiting loop")
        print(f"Error: {e}")
        break

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df.head()


In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].dropna().to_list()
print("Preview of Titles List:")
for title in titles[:10]:  # Print the first 20 titles for readability
    print(f"'{title}'")


In [ ]:
### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
# Directly set the TMDb API key string
# Set the TMDb API key and base URLs
tmdb_search_url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_movie_url = "https://api.themoviedb.org/3/movie/"
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the TMDb results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 1
# Loop through the titles
# Check if we need to sleep before making a request    
titles = reviews_df['title'].dropna().to_list()

for title in titles:
    if request_counter % 50 == 0:
        print("Sleeping for 60 seconds to avoid hitting rate limits...")
        time.sleep(60)
        
# Perform a "GET" request for The Movie Database    
# Build the search query URL
    search_query_url = f"{tmdb_search_url}{title}{tmdb_key_string}"
    search_response = requests.get(search_query_url)

   # Add 1 to the request counter
    
    request_counter += 1


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    
    try:
        search_data = search_response.json()
        if not search_data['results']:
            print(f"No movie found for title: {title}")
            continue
        
        # Get movie id
        movie_id = search_data['results'][0]['id']
        
        # Build the movie details query URL
        movie_query_url = f"{tmdb_movie_url}{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_query_url)
        movie_data = movie_response.json()
        
        genres = [genre['name'] for genre in movie_data.get('genres', [])]
        spoken_languages = [lang['english_name'] for lang in movie_data.get('spoken_languages', [])]
        production_countries = [country['name'] for country in movie_data.get('production_countries', [])]
        
        movie_info = {
            'title': title,
            'original_title': movie_data.get('original_title', ''),
            'budget': movie_data.get('budget', ''),
            'original_language': movie_data.get('original_language', ''),
            'homepage': movie_data.get('homepage', ''),
            'overview': movie_data.get('overview', ''),
            'popularity': movie_data.get('popularity', ''),
            'runtime': movie_data.get('runtime', ''),
            'revenue': movie_data.get('revenue', ''),
            'release_date': movie_data.get('release_date', ''),
            'vote_average': movie_data.get('vote_average', ''),
            'vote_count': movie_data.get('vote_count', ''),
            'genres': ", ".join(genres),
            'spoken_languages': ", ".join(spoken_languages),
            'production_countries': ", ".join(production_countries)
        }
        tmdb_movies_list.append(movie_info)
        print(f"Successfully retrieved details for title: {title}")
    except Exception as e:
        print(f"Failed to retrieve details for title: {title}")
        print(f"Error: {e}")


In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

In [ ]:
#Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print("Preview of TMDb Data:")
print(json.dumps(tmdb_movies_list[:5], indent=4))

In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

In [ ]:
### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_df, on='title', how='inner')

# Print the head of the merged DataFrame to check the result
print("Preview of Merged Data:")
print(merged_df.head())

# Verify the merge by checking for the presence of key columns
print("Columns in the merged DataFrame:")
print(merged_df.columns)

# Check for unexpected NaN values
print("Number of NaN values in each column:")
print(merged_df.isnull().sum())

In [ ]:
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Ensure the column contains string representations of lists
    merged_df[column] = merged_df[column].astype(str)
    
    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, "", regex=False)

# Display the fixed DataFrame
print("Fixed Merged DataFrame:")
print(merged_df.head())

In [ ]:
# Drop the "byline.person" column if it exists
columns_to_drop = [col for col in merged_df.columns if 'byline.person' in col]
if columns_to_drop:
    merged_df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
print("Columns in the merged DataFrame after attempting to drop specified columns:")
print(merged_df.columns)

In [ ]:
 # Remove duplicate rows and reset the index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)

In [ ]:
# Export data to CSV without the index
merged_df.to_csv('merged_cleaned_data.csv', index=False)